### Prototype data Precessing with Spacy

1. Import model
2. Import database
3. iterate through database and label the data
4. Create DF with labeled data as features
5. Export DF as 'results.csv'

In [1]:
import spacy
import pandas as pd 
import numpy as np 

In [2]:
# Loading the Model

nlp = spacy.load("../training/test_model")

In [3]:
# Loading the input_data

data = pd.read_csv("input_data_filtered.csv")

In [4]:
data["PrimaryOutcomeMeasure"] = data["PrimaryOutcomeMeasure"].astype(str)
data["PrimaryTimeFrame"] = data["PrimaryTimeFrame"].astype(str)
data["SecondaryOutcomeMeasure"] = data["SecondaryOutcomeMeasure"].astype(str)
data["SecondaryTimeFrame"] = data["SecondaryTimeFrame"].astype(str)

In [5]:
data_result_df = pd.DataFrame(columns=["NCTId", "Condition", 'Change', "Reference", "Variable","Timepoint"])

In [6]:
data_result_df

,NCTId,Condition,Change,Reference,Variable,Timepoint


In [7]:
# Iteration through data 
# TODO: Remove Measurment-Title ???



def data_to_dataframe(data):
    dfTest = pd.DataFrame(columns=["NCTId","Condition","Change","Reference", "Variable", "Timepoint"])

    for index, row in data.iterrows():
        pointer = 0
        trial_id = row[1]

        condition = row[2]
        
        primary_endpoint = row[3]
        primary_endpoints = []

        primary_endpoint_timepoint = row[4]
        primary_endpoint_timepoints = []

        secondary_endpoint = row[5]
        secondary_endpoints = []

        secondary_endpoint_timepoint = row[6]
        secondary_endpoint_timepoints = []
        
        if  primary_endpoint.find("|") != -1:
            primary_endpoints = primary_endpoint.split("|")

        if  primary_endpoint_timepoint.find("|") != -1:
            primary_endpoint_timepoints = primary_endpoint_timepoint.split("|")

        if  secondary_endpoint.find("|") != -1:
            secondary_endpoints = secondary_endpoint.split('|')

        if  secondary_endpoint_timepoint.find("|") != -1:
            secondary_endpoint_timepoints = secondary_endpoint_timepoint.split('|')
        
        if primary_endpoints == []:
            endpoint_art = "Primary Endpoint"
            dfTemp = process_text_entry(trial_id, condition, primary_endpoint, endpoint_art, pointer, primary_endpoint_timepoint)
            dfTest = dfTest.append(dfTemp, ignore_index= True)
        else:
            for point, timepoint in zip(primary_endpoints, primary_endpoint_timepoints):
                endpoint_art = "Primary Endpoint"
                dfTemp = process_text_entry(trial_id, condition, point, endpoint_art, pointer, timepoint)
                dfTest = dfTest.append(dfTemp, ignore_index= True)


        if secondary_endpoints == []:
            endpoint_art="Secondary Endpoint"
            dfTemp = process_text_entry(trial_id, condition, secondary_endpoint, endpoint_art, pointer, secondary_endpoint_timepoint)
            dfTest = dfTest.append(dfTemp, ignore_index= True)
        else:
            for point, timepoint in zip(secondary_endpoints, secondary_endpoint_timepoints):
                endpoint_art = "Secondary Endpoint"
                dfTemp = process_text_entry(trial_id, condition, point,endpoint_art, pointer, timepoint)
                dfTest = dfTest.append(dfTemp, ignore_index= True)
                pointer+=1

    return dfTest


In [8]:
# TODO: If multiple equal labels they must be concatinated 
def process_text_entry(nctid, condition, text, endpoint_art, pointer, timepoint):
    dfTemp = pd.DataFrame(columns=["NCTId","Condition","Change","Reference", "Variable", "Timepoint", "Endpoint Art"])

    target = nlp(text)
    for entity in target.ents:
        dfTemp.at[pointer,"NCTId"]=nctid
        dfTemp.at[pointer,"Condition"]=condition
        if entity.label_ != "Condition":
            dfTemp[entity.label_]=entity.text
        dfTemp.at[pointer,"Endpoint Art"]=endpoint_art
        if entity.label_ != "Timepoint":
            dfTemp.at[pointer, "Timepoint"] = timepoint
    return dfTemp


In [9]:

results_df = data_to_dataframe(data)

In [10]:
results_df

,NCTId,Condition,Change,Reference,Variable,Timepoint,Endpoint Art
0,NCT01814787,Type 2 Diabetes,Number of Children,NaN,Documented Risk Factors,12 months,Primary Endpoint
1,NCT01346033,Type 2 Diabetes,NaN,Validation,SCOUT DS algorithm,1 day,Primary Endpoint
2,NCT04765631,Type 2 Diabetes,outcome,comparison,total volumetric bone mineral density (vBMD),at inclusion for Qualyor patients. (the HRpQCT...,Primary Endpoint
3,NCT03979768,Type2 Diabetes,Number of Participants,NaN,NaN,2 months,Primary Endpoint
4,NCT03979768,Type2 Diabetes,number of participants,NaN,HbA1c-measurement (HbA1c-group),Baseline,Secondary Endpoint
...,...,...,...,...,...,...,...
311,NCT01755468,Type 2 Diabetes,NaN,Baseline,NaN,2-years,Secondary Endpoint
312,NCT04363710,Type 2 Diabetes Mellitus in Obese,NaN,percentage,HbA1c,up to 24 weeks,Primary Endpoint
313,NCT04363710,Type 2 Diabetes Mellitus in Obese,NaN,liter,Fasting Insulin Level,up to 24 weeks,Primary Endpoint
314,NCT04363710,Type 2 Diabetes Mellitus in Obese,NaN,BMI,Weight,up to 24 weeks,Primary Endpoint


In [11]:
results_df.index

RangeIndex(start=0, stop=316, step=1)

In [12]:
results_df = results_df[(results_df["Change"] != "Not Mentioned") & (results_df["Variable"] != "Not Mentioned") &
                        (results_df["Reference"] != "Not Mentioned") & (results_df["Timepoint"] != "Not Mentioned")]

In [13]:
export = results_df
export["Condition"] = export["Condition"].str.title()
export["Reference"] = export["Reference"].str.title()
export["Variable"] = export["Variable"].str.title()
export["Timepoint"] = export["Timepoint"].str.title()
export["Change"] = export["Change"].str.title()

export = export.fillna('Not Mentioned')
export.rename_axis("Index", axis='index', inplace=True)
export.to_csv("results.csv")